In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from collections import defaultdict
from multiprocessing import Pool
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

from generator import generate_tracks_paper, generate_tracks_time_varying, generate_tracks_sudden_birth
from runners import generate_tests, run_test_from_config
from test_case import TestUseCase2D

from gmphd_fusion.data import StateVector, CovarianceMatrix
from gmphd_fusion.gm import Gaussian, GaussianMixture
from gmphd_fusion.measurement_model import LinearCoordinateMeasurementModel
from gmphd_fusion.motion_models import ConstantVelocityMotionModel

In [3]:
plt.style.use("bmh")
plt.rcParams.update({"figure.figsize": (24, 12),
                     "axes.facecolor": "white",
                     "axes.edgecolor": "black"})

# Define testing use cases

In [4]:
N_RUNS = 100
EXPERIMENTS_DIR = Path("../experiments")

# Change of parameters that we are going to measure
TEST_PARAMS = dict(
    clutter_rate=[5. * (i + 1) for i in range(10)],
    detection_prob=[0.7 + 0.05 * i for i in range(7)],
)

TEST_CASES = dict(
    two_obj_cross=TestUseCase2D(
        surveillance_region=((-1000., -1000.), (1000., 1000.)),
        clutter_rate=50,
        detection_prob=0.98,
        survival_prob=0.99,
        prune_threshold=1e-5,
        merge_threshold=4,
        max_components=1000,
        init_gm=GaussianMixture(),
        birth_gm=GaussianMixture(
            gaussians=[
                Gaussian(
                    mean=StateVector([250, 250, 0, 0]),
                    cov=CovarianceMatrix(np.diag([100, 100, 25, 25])),
                    label=Gaussian.BIRTH_LABEL),
                Gaussian(
                    mean=StateVector([-250, -250, 0, 0]),
                    cov=CovarianceMatrix(np.diag([100, 100, 25, 25])),
                    label=Gaussian.BIRTH_LABEL),
            ],
            weights=[0.1, 0.1],
        ),
        motion_model=ConstantVelocityMotionModel(motion_noise=5),
        measurement_model=LinearCoordinateMeasurementModel(dim_measurement=2, dim_state=4, measurement_noise=10),
        tracks_true=generate_tracks_paper(),
        cpep_radius=20,
        target_weight_threshold=0.5,
    ),
    birth_death_vary=TestUseCase2D(
        surveillance_region=((-1000., -1000.), (1000., 1000.)),
        clutter_rate=50,
        detection_prob=0.98,
        survival_prob=0.99,
        prune_threshold=1e-5,
        merge_threshold=4,
        max_components=1000,
        init_gm=GaussianMixture(),
        birth_gm=GaussianMixture(
            gaussians=[
                Gaussian(
                    mean=StateVector([-1000, 750, 0, 0]),
                    cov=CovarianceMatrix(np.diag([100, 100, 30, 30])),
                    label=Gaussian.BIRTH_LABEL),
                Gaussian(
                    mean=StateVector([1000, -750, 0, 0]),
                    cov=CovarianceMatrix(np.diag([100, 100, 30, 30])),
                    label=Gaussian.BIRTH_LABEL),
            ],
            weights=[0.1, 0.1],
        ),
        motion_model=ConstantVelocityMotionModel(motion_noise=5),
        measurement_model=LinearCoordinateMeasurementModel(dim_measurement=2, dim_state=4, measurement_noise=10),
        tracks_true=generate_tracks_time_varying(),
        cpep_radius=20,
        target_weight_threshold=0.5,
    ),
    birth_no_fusion=TestUseCase2D(
        surveillance_region=((-1000., -1000.), (1000., 1000.)),
        clutter_rate=50,
        detection_prob=0.98,
        survival_prob=0.99,
        prune_threshold=1e-5,
        merge_threshold=4,
        max_components=1000,
        init_gm=GaussianMixture(),
        birth_gm=GaussianMixture(
            gaussians=[
                Gaussian(
                    mean=StateVector([-1000, 750, 0, 0]),
                    cov=CovarianceMatrix(np.diag([100, 100, 30, 30])),
                    label=Gaussian.BIRTH_LABEL),
                Gaussian(
                    mean=StateVector([1000, -750, 0, 0]),
                    cov=CovarianceMatrix(np.diag([100, 100, 30, 30])),
                    label=Gaussian.BIRTH_LABEL),
            ],
            weights=[0.1, 0.1],
        ),
        motion_model=ConstantVelocityMotionModel(motion_noise=5),
        measurement_model=LinearCoordinateMeasurementModel(dim_measurement=2, dim_state=4, measurement_noise=10),
        tracks_true=generate_tracks_sudden_birth(),
        cpep_radius=20,
        target_weight_threshold=0.5,
    ),
    birth_fusion=TestUseCase2D(
        surveillance_region=((-1000., -1000.), (1000., 1000.)),
        clutter_rate=50,
        detection_prob=0.98,
        survival_prob=0.99,
        prune_threshold=1e-5,
        merge_threshold=4,
        max_components=1000,
        init_gm=GaussianMixture(),
        birth_gm=GaussianMixture(
            gaussians=[
                Gaussian(
                    mean=StateVector([-1000, 750, 0, 0]),
                    cov=CovarianceMatrix(np.diag([100, 100, 30, 30])),
                    label=Gaussian.BIRTH_LABEL),
                Gaussian(
                    mean=StateVector([1000, -750, 0, 0]),
                    cov=CovarianceMatrix(np.diag([100, 100, 30, 30])),
                    label=Gaussian.BIRTH_LABEL),
            ],
            weights=[0.1, 0.1],
        ),
        motion_model=ConstantVelocityMotionModel(motion_noise=5),
        measurement_model=LinearCoordinateMeasurementModel(dim_measurement=2, dim_state=4, measurement_noise=10),
        tracks_true=generate_tracks_sudden_birth(),
        fuse={
            20: GaussianMixture(
                gaussians=[
                    Gaussian(StateVector([-1050, -1050, 25, 25]), cov=CovarianceMatrix(np.diag([100, 100, 10, 10])),
                             label=None)],
                weights=[0.6],
            ),
            60: GaussianMixture(
                gaussians=[Gaussian(StateVector([1020, 1020, -22, -22]), cov=CovarianceMatrix(np.diag([40, 40, 5, 5])),
                                    label=None)],
                weights=[0.7],
            ),
        },
        cpep_radius=20,
        target_weight_threshold=0.5,
    ),
)

# Run

In [5]:
RUN = False

run_configurations = generate_tests(N_RUNS, TEST_CASES, TEST_PARAMS, EXPERIMENTS_DIR)
print(f"Total number of test configurations: {len(run_configurations)}")

Total number of test configurations: 6800


In [6]:
%%time
if RUN:
    pool = Pool()
    success_flags = pool.map(run_test_from_config, run_configurations)
else:
    success_flags = [True] * len(run_configurations)

failed_indices = np.where(~np.array(success_flags))[0]
print(f"Total failed: {len(failed_indices)}")

Total failed: 0
CPU times: user 964 ms, sys: 732 ms, total: 1.7 s
Wall time: 16h 56min 25s


# Compute metrics and build box plots

In [ ]:
def prepare_metrics_struct():
    struct = {}
    for tname in TEST_CASES.keys():
        struct[tname] = dict(
            cpep={
                param_name: {
                    f"{param_val:.3f}": [0.] * N_RUNS
                    for param_val in TEST_PARAMS[param_name]
                }
                for param_name in TEST_PARAMS.keys()
            },
            eae={
                param_name: {
                    f"{param_val:.3f}": [0.] * N_RUNS
                    for param_val in TEST_PARAMS[param_name]
                }
                for param_name in TEST_PARAMS.keys()
            }
        )
    return struct


def _read_metrics(p: Path):
    with (p / "cpep_time.pickle").open("rb") as f:
        cpep = pickle.load(f)
        cpep = np.mean(cpep)
    with (p / "eae.pickle").open("rb") as f:
        eae = pickle.load(f)
    return cpep, eae


def read_saved_metrics(experiments_dir: Path):
    paths = sorted([p.parent for p in experiments_dir.glob("**/_FINISHED")])
    
    metrics_data = prepare_metrics_struct()
    for p in paths:
        # path structure: {test_name}/{param_name}={param_value:.3f}/{index:03d}
        index = int(p.name)
        param_name, param_value = p.parent.name.split("=")
        test_name = p.parent.parent.name
        
        cpep, eae = _read_metrics(p)
        metrics_data[test_name]["cpep"][param_name][param_value][index] = cpep
        metrics_data[test_name]["eae"][param_name][param_value][index] = eae
    
    ret = {}
    for test_name, metrics_dict in metrics_data.items():
        for metric_name, params_dict in metrics_dict.items():
            for param_name, param_values in params.items():
                param_values, runs_data = zip(*sorted(param_values.items(), key=lambda t: t[0]))
                ret[(test_name, metric_name, param_name)] = {
                    "labels": param_values,
                    "data": runs_data,
                }
    return ret

In [ ]:
metrics_mapping = {
    "cpep": "$\mathrm{CPEP}_k(r) (\text{time averaged})$",
    "eae": "$\mathbb{E}\left{\lvert \lvert\hat{X}_k\rvert - \lvert\X_k\rvert\rvert\right} (\text{time averaged})$",
}
param_mapping = {
    "clutter_rate": {
        "label_func": lambda l: float(l) / 4e6,
        "x_label": "$\lambda_c$",
    },
    "detection_prob": {
        "label_func": lambda l: float(l),
        "x_label": "$p_{D,k}$",
    }
}


metrics_data = read_saved_metrics(EXPERIMENTS_DIR)

for (test_name, metric_name, param_name), mdata in metrics_data.items():
    x_label = param_mapping[param_name]["x_label"]
    y_label = metrics_mapping[metric_name]
    x_ticks = [param_mapping[param_name]["label_func"](l) for l in mdata["labels"]]
    experiments = mdata["data"]
    save_dir = EXPERIMENTS_DIR / test_name
    save_box_whisker_plot(x_label, y_label, x_ticks, experiments, save_dir)